<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/re_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import numpy as np
import pandas as pd
import re

In [51]:
file_path = 'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/FantasyPros_2024_Overall_ADP_Rankings.csv'
df = pd.read_csv(file_path)#, on_bad_lines='skip')
df['POS'] = df['POS'].str.replace('\d+', '', regex=True)

def extract_numbers(s):
    if pd.isna(s):
        return None
    numbers = re.findall(r'\d+', str(s))
    if numbers:
        return int(numbers[0])
    return None

df_small = df[['Player', 'Bye', 'POS', 'AVG']].head(250).copy()
df_small['Bye'] = df_small['Bye'].apply(extract_numbers)
df_small['Bye'] = df_small['Bye'].fillna(0).astype(int)

df_small.info()
print(df_small.value_counts('POS'))
print(df_small.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  250 non-null    object 
 1   Bye     250 non-null    int64  
 2   POS     250 non-null    object 
 3   AVG     250 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 7.9+ KB
POS
WR     77
RB     66
QB     31
DST    28
TE     28
K      20
Name: count, dtype: int64
                 Player  Bye POS   AVG
0   Christian McCaffrey    9  RB   1.0
1           CeeDee Lamb    7  WR   2.6
2           Tyreek Hill    6  WR   3.2
3        Bijan Robinson   12  RB   5.0
4           Breece Hall   12  RB   5.4
5     Amon-Ra St. Brown    5  WR   6.2
6         Ja'Marr Chase   12  WR   6.6
7      Justin Jefferson    6  WR   7.0
8        Saquon Barkley    5  RB   9.2
9            A.J. Brown    5  WR  10.2
10      Jonathan Taylor   14  RB  10.4
11       Garrett Wilson   12  WR  12.4
12         Jahm

In [52]:
# Anzahl der Wochen
number_of_weeks = 17

# Wochen-Spaltennamen
weekly_columns = [f"Week_{i+1}" for i in range(number_of_weeks)]

# Skalierungsfunktion
def final_projection_base(avg, pos, max_val=22, min_val=7, k=50, c=1.5):
    base = min_val + (max_val - min_val) * (1 / (1 + (avg / k) ** c))
    if pos == "QB":
        base += 4  # QB-Bonus
    return base

final_projections = []
for _, row in df_small.iterrows():
    base_score = final_projection_base(row['AVG'], row['POS'])

    # Erstellen der weekly projections
    weekly_proj = []
    for week in range(number_of_weeks):
        # Überprüfen, ob die aktuelle Woche (week + 1) mit der Bye-Woche des Spielers übereinstimmt
        if (week + 1) == row['Bye']:  # Woche des Spielers = Bye-Woche?
            weekly_proj.append(0.0)  # Projektion auf 0 setzen
        else:
            weekly_proj.append(base_score + np.random.normal(0, base_score * 0.1))  # Zufällige Variation

    final_projections.append(weekly_proj)

# Projektionen in DataFrame einfügen
f = df_small[["Player", "Bye", "POS", "AVG"]].copy()
for i, col in enumerate(weekly_columns):
    f[col] = [proj[i] for proj in final_projections]

# Zeige eine zufällige Stichprobe der ersten 25 Zeilen
f.head(25)


,Player,Bye,POS,AVG,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,...,Week_8,Week_9,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17
0,Christian McCaffrey,9,RB,1.0,22.114081,21.248396,20.918577,23.791382,23.354110,24.119934,...,21.727374,0.000000,20.822613,21.313989,25.279119,21.245517,19.505598,22.610512,23.605370,23.046533
1,CeeDee Lamb,7,WR,2.6,22.074412,19.421313,23.350946,22.193700,21.588507,23.636405,...,15.411639,18.448065,21.970411,22.345250,24.922013,22.169791,23.920739,21.682774,23.698085,21.673983
2,Tyreek Hill,6,WR,3.2,19.757462,23.499867,24.452596,20.702928,24.560964,0.000000,...,21.547094,23.563341,22.970855,22.080077,23.774045,21.434910,22.100083,25.607513,25.101277,16.458855
3,Bijan Robinson,12,RB,5.0,18.500633,18.814639,17.783973,22.249618,19.996911,19.695222,...,19.341481,23.371583,24.729446,20.537508,0.000000,25.496410,22.234999,21.255332,19.483813,20.431838
4,Breece Hall,12,RB,5.4,21.666421,18.654646,18.506726,21.165200,22.103943,19.734470,...,22.338615,19.026737,19.576883,22.737993,0.000000,19.303386,25.613094,21.674341,26.232197,23.384344
5,Amon-Ra St. Brown,5,WR,6.2,21.103750,23.400408,22.496696,20.452838,0.000000,21.316938,...,20.110922,17.241296,20.466951,18.084123,20.551492,22.528252,20.218232,18.613831,21.620775,21.367711
6,Ja'Marr Chase,12,WR,6.6,18.689306,20.710652,19.457201,21.872755,20.947681,15.669723,...,22.349468,17.868905,20.979795,20.583989,0.000000,23.066686,20.811598,22.283923,21.708254,17.212703
7,Justin Jefferson,6,WR,7.0,21.481529,22.471470,23.699095,23.264800,17.655830,0.000000,...,25.323001,18.624202,21.417692,19.639410,21.460905,23.344652,18.566157,21.388932,19.838879,24.113078
8,Saquon Barkley,5,RB,9.2,23.305682,20.200595,20.236818,22.248983,0.000000,21.370527,...,19.571831,24.706053,22.590850,20.687216,25.847940,20.834638,23.932247,22.420168,16.363070,18.942134
9,A.J. Brown,5,WR,10.2,19.563792,22.779185,20.437584,21.269339,0.000000,23.157045,...,19.747735,21.715736,24.014778,20.695715,22.954320,16.111732,22.326106,19.174901,17.796967,21.327081


In [53]:
week_cols = [col for col in f.columns if col.startswith("Week_")]

# Dictionary: f_dict[player][week] = projection
f_dict = {
    row['Player']: {int(week.replace("Week_", "")): row[week] for week in week_cols}
    for _, row in f.iterrows()
}
list(f_dict.items())[:3]


[('Christian McCaffrey',
  {1: 22.114081334060213,
   2: 21.24839643378607,
   3: 20.91857708878705,
   4: 23.791381698621205,
   5: 23.354109757277858,
   6: 24.11993402292733,
   7: 25.646888520681998,
   8: 21.727374207894787,
   9: 0.0,
   10: 20.822613207380286,
   11: 21.31398939758424,
   12: 25.279118699594605,
   13: 21.24551676892532,
   14: 19.505597915252118,
   15: 22.610511973031688,
   16: 23.605370484093562,
   17: 23.04653313479207}),
 ('CeeDee Lamb',
  {1: 22.074412431724767,
   2: 19.421313241468,
   3: 23.35094571931655,
   4: 22.19370048847041,
   5: 21.588507371205367,
   6: 23.63640511385961,
   7: 0.0,
   8: 15.411638719576693,
   9: 18.44806486738691,
   10: 21.97041066915078,
   11: 22.345249831057586,
   12: 24.922012883251117,
   13: 22.169791460822022,
   14: 23.920738616724513,
   15: 21.68277353258832,
   16: 23.698084735860853,
   17: 21.67398332118107}),
 ('Tyreek Hill',
  {1: 19.75746222496911,
   2: 23.49986667698006,
   3: 24.452596055757116,
   4: 2

In [54]:
week_cols = [col for col in f.columns if col.startswith("Week_")]
f_dict = {
    row['Player']: {int(week.replace("Week_", "")): row[week] for week in week_cols}
    for _, row in f.iterrows()
}
list(f_dict.items())[:3]

[('Christian McCaffrey',
  {1: 22.114081334060213,
   2: 21.24839643378607,
   3: 20.91857708878705,
   4: 23.791381698621205,
   5: 23.354109757277858,
   6: 24.11993402292733,
   7: 25.646888520681998,
   8: 21.727374207894787,
   9: 0.0,
   10: 20.822613207380286,
   11: 21.31398939758424,
   12: 25.279118699594605,
   13: 21.24551676892532,
   14: 19.505597915252118,
   15: 22.610511973031688,
   16: 23.605370484093562,
   17: 23.04653313479207}),
 ('CeeDee Lamb',
  {1: 22.074412431724767,
   2: 19.421313241468,
   3: 23.35094571931655,
   4: 22.19370048847041,
   5: 21.588507371205367,
   6: 23.63640511385961,
   7: 0.0,
   8: 15.411638719576693,
   9: 18.44806486738691,
   10: 21.97041066915078,
   11: 22.345249831057586,
   12: 24.922012883251117,
   13: 22.169791460822022,
   14: 23.920738616724513,
   15: 21.68277353258832,
   16: 23.698084735860853,
   17: 21.67398332118107}),
 ('Tyreek Hill',
  {1: 19.75746222496911,
   2: 23.49986667698006,
   3: 24.452596055757116,
   4: 2

In [55]:
print(f_dict['Christian McCaffrey'][9])
print(f_dict['CeeDee Lamb'][17])


0.0
21.67398332118107


In [56]:
!pip install mip

In [57]:
import pandas as pd
from mip import Model, BINARY, maximize, xsum

# -- Annahme: df ist bereits geladen und enthält die Spalten wie beschrieben --

# 1. Spieler-Liste und Positionszuordnung
players = df_small['Player'].tolist()
pos = dict(zip(df_small['Player'], df_small['POS']))

# 3. Weitere Parameter
weeks = list(range(1, 18))
positions = {"QB", "RB", "WR", "TE", "K", "DST"}
gamma = {"QB": 2, "RB": 5, "WR": 6, "TE": 2, "K": 1, "DST": 2}
pos_limit = {"QB": 1, "RB": 2, "WR": 2, "TE": 1, "K": 1, "DST": 1}
beta = {t: 100.0 for t in weeks}  # Beispielhafte Gewinnschwelle

# 4. Beispielhafte Listen für bereits gedraftete Spieler (hier leer)
dm_players = []
opp_players = []

# 5. Initialisierung des Modells
model = Model("FantasyFootball", sense=maximize)

# Entscheidungsvariablen
y = {i: model.add_var(name=f"y_{i}", var_type=BINARY) for i in players}
x = {(i, t): model.add_var(name=f"x_{i}_{t}") for i in players for t in weeks}
z = {t: model.add_var(name=f"z_{t}", var_type=BINARY) for t in weeks}

# Zielfunktion
model.objective = (
    1.0 * xsum(f_dict[i][t] * x[i, t] for i in players for t in weeks) +
    100.0 * xsum(z[t] for t in range(1, 16)) +
    150.0 * xsum(z[t] for t in range(16, 18))
)

# Constraints
for j in positions:
    # Mindestanzahl gedrafteter Spieler pro Position (1c)
    model += xsum(y[i] for i in players if pos[i] == j) >= gamma[j]
    # Positionslimits pro Woche (1d)
    for t in weeks:
        model += xsum(x[i, t] for i in players if pos[i] == j) <= pos_limit[j]

model += xsum(y[i] for i in players) <= 18

# Logische Verknüpfung Draft ↔ Aufstellung (1e)
for i in players:
    for t in weeks:
        model += x[i, t] <= y[i]

# Gewinnbedingung (1f)
for t in weeks:
    model += z[t] <= xsum(f_dict[i][t] * x[i, t] for i in players) / beta[t]

# Bereits gedraftete Spieler (1g-1h)
for i in dm_players:
    model += y[i] == 1
for i in opp_players:
    model += y[i] == 0

# Optimierung starten
model.optimize()


<OptimizationStatus.OPTIMAL: 0>

In [58]:
drafted_players = [i for i in players if y[i].x >= 0.99]
print("Dein Team:")
for player in drafted_players:
    print(player)

# Zielfunktionswert (Gesamtpunkte)
print("Zielfunktionswert:", model.objective_value)

Dein Team:
Kyle Pitts
Chase Brown
New York Jets
Matthew Stafford
Cameron Dicker
Luke Musgrave
Antonio Gibson
Bucky Irving
Josh Downs
Ray Davis
Tyrone Tracy Jr.
Minnesota Vikings
Jahan Dotson
Jermaine Burton
Drake Maye
Luke McCaffrey
Marvin Mims Jr.
Wan'Dale Robinson
Zielfunktionswert: 0.0


In [79]:
import pandas as pd
from mip import Model, BINARY, maximize, xsum

### Parameter ###
# The set of NFL players and defensive teams.
players = df_small['Player'].tolist()

# The set of positions
positions = {"QB", "RB", "WR", "TE", "K", "DST"}

# The set of weeks in the NFL regular and playoff seasons
weeks = list(range(1, 18))

# Players and their position
pos = dict(zip(df_small['Player'], df_small['POS']))

# Position Limit: The upper bound on the number of starting players for position
pos_limit = {"QB": 1, "RB": 2, "WR": 2, "TE": 1, "K": 1, "DST": 1}

# The overall pick number of the DM’s k-th draft pick
n_k = 0

# The set of players that the DM has drafted by her k-th pick
dm_players = []

# The set of players that the Opponents have drafted by her k-th pick
opp_players = []

gamma = {"QB": 1, "RB": 2, "WR": 2, "TE": 1, "K": 1, "DST": 1}
beta = {t: 100.0 for t in weeks}  # Beispielhafte Gewinnschwelle

# Teams und Snake-Draft-Reihenfolge
number_of_teams = 6
teams = [f"Team {i+1}" for i in range(number_of_teams)]
num_rounds = 15

# Snake Draft Reihenfolge erzeugen
draft_order = []
for r in range(num_rounds):
    if r % 2 == 0:
        draft_order.extend(teams)
    else:
        draft_order.extend(teams[::-1])

# Initialisiere leere Roster und verfügbare Spieler
rosters = {team: [] for team in teams}
available_players = set(players)
draft_log = []

for pick_num, team in enumerate(draft_order):
    model = Model(sense=maximize, solver_name="CBC")

    # Entscheidungsvariablen
    y = {i: model.add_var(name=f"y_{i}", var_type=BINARY) for i in players}
    x = {(i, t): model.add_var(name=f"x_{i}_{t}") for i in players for t in weeks}
    z = {t: model.add_var(name=f"z_{t}", var_type=BINARY) for t in weeks}

    # Bereits gedraftete Spieler verbieten
    for i in players:
        if i not in available_players:
            model += y[i] == 0

    # Bereits im Team befindliche Spieler erzwingen
    for i in rosters[team]:
        model += y[i] == 1

    # Zielfunktion
    model.objective = (
        xsum(f_dict[i][t] * x[i, t] for i in players for t in weeks) +
        100.0 * xsum(z[t] for t in range(1, 16)) +
        150.0 * xsum(z[t] for t in range(16, 18))
    )

    # Constraints
    for j in positions:
        model += xsum(y[i] for i in players if pos[i] == j) >= gamma[j]
        for t in weeks:
            model += xsum(x[i, t] for i in players if pos[i] == j) <= pos_limit[j]

    # Rostergröße (Passe ggf. an!)
    model += xsum(y[i] for i in players) <= 12

    for i in players:
        for t in weeks:
            model += x[i, t] <= y[i]

    for t in weeks:
        model += z[t] <= xsum(f_dict[i][t] * x[i, t] for i in players) / beta[t]

    # Optimierung
    status = model.optimize()

    # Spieler auswählen, der neu gedraftet wird
    picked_player = None
    for i in players:
        if y[i].x is not None and y[i].x >= 0.99 and i not in rosters[team] and i in available_players:
            picked_player = i
            break

    if picked_player is None:
        print(f"Kein Spieler für Team {team} gefunden bei Pick {pick_num + 1}")
        break

    # Spieler zum Roster hinzufügen und aus verfügbaren Spielern entfernen
    rosters[team].append(picked_player)
    available_players.remove(picked_player)

    # Loggen
    round_num = pick_num // len(teams) + 1
    draft_log.append({"Pick": pick_num + 1, "Team": team, "Player": picked_player, "Round": round_num})

# DataFrame aus Log erzeugen
df_draft = pd.DataFrame(draft_log)
df_draft


Kein Spieler für Team Team 6 gefunden bei Pick 7


,Pick,Team,Player,Round
0,1,Team 1,Chase Brown,1
1,2,Team 2,De'Von Achane,1
2,3,Team 3,Josh Jacobs,1
3,4,Team 4,Tyjae Spears,1
4,5,Team 5,Travis Kelce,1
5,6,Team 6,Lamar Jackson,1
